In [ ]:
# gtx5070ti를 지원하는 cuda12.8버전 지정 설치
#pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu128

In [ ]:
# import torch

# if torch.cuda.is_available():
#     print("GPU is available!")
#     print(f"Device name: {torch.cuda.get_device_name(0)}")
# else:
#     print("GPU is not available.")

GPU is available!
Device name: NVIDIA GeForce RTX 5070 Ti


In [ ]:
# 1.기본 라이브러리 및 환경 설정

# 라이브러리 임포트
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# GPU 사용 가능 여부 확인 및 장치 설정
# 현재 GPU 호환성 문제로 CPU를 사용하도록 고정합니다.
device = torch.device("cpu")
print(f"Using device: {device}")

# 하이퍼파라미터 설정
batch_size = 64
learning_rate = 0.001
epochs = 10

Using device: cpu
--- Epoch 1/10 ---
Train Epoch: [0/60000 (0%)]	Loss: 2.305836
Train Epoch: [6400/60000 (11%)]	Loss: 0.426271
Train Epoch: [12800/60000 (21%)]	Loss: 0.286233
Train Epoch: [19200/60000 (32%)]	Loss: 0.352440
Train Epoch: [25600/60000 (43%)]	Loss: 0.227961
Train Epoch: [32000/60000 (53%)]	Loss: 0.279167
Train Epoch: [38400/60000 (64%)]	Loss: 0.153687
Train Epoch: [44800/60000 (75%)]	Loss: 0.342793
Train Epoch: [51200/60000 (85%)]	Loss: 0.141994
Train Epoch: [57600/60000 (96%)]	Loss: 0.258028

Test set: Average loss: 0.0027, Accuracy: 9482/10000 (95%)

--- Epoch 2/10 ---
Train Epoch: [0/60000 (0%)]	Loss: 0.102550
Train Epoch: [6400/60000 (11%)]	Loss: 0.059432
Train Epoch: [12800/60000 (21%)]	Loss: 0.056525
Train Epoch: [19200/60000 (32%)]	Loss: 0.118420
Train Epoch: [25600/60000 (43%)]	Loss: 0.075792
Train Epoch: [32000/60000 (53%)]	Loss: 0.203417
Train Epoch: [38400/60000 (64%)]	Loss: 0.049665
Train Epoch: [44800/60000 (75%)]	Loss: 0.062971
Train Epoch: [51200/60000 (85%)

In [ ]:
# 2.데이터 로더 준비

# 데이터 전처리 및 로더 정의
transform = transforms.ToTensor()

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)




In [ ]:
# 3.CNN 모델 정의

# 신경망 모델 정의 (CNN 버전)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # 첫 번째 합성곱 레이어
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2)
        # 두 번째 합성곱 레이어
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2)
        # 완전 연결 레이어
        self.fc1 = nn.Linear(in_features=7*7*32, out_features=10)

    def forward(self, x):
        # 첫 번째 합성곱 레이어 -> ReLU -> 최대 풀링
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        # 두 번째 합성곱 레이어 -> ReLU -> 최대 풀링
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        # 텐서를 1차원으로 펼치기
        x = x.view(-1, 7*7*32)
        # 완전 연결 레이어
        x = self.fc1(x)
        return x

In [ ]:
# 4.모델, 손실 함수, 옵티마이저 설정

# 모델 인스턴스 생성 및 장치로 이동
model = CNN().to(device)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# 5.학습 및 평가 함수 정의

# 학습 루프 구현
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: [{batch_idx * len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

# 평가 함수 구현
def evaluate(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n')

In [ ]:
# 6.메인 학습 루프 실행 및 모델 저장

# 8. 메인 함수
if __name__ == "__main__":
    for epoch in range(1, epochs + 1):
        print(f"--- Epoch {epoch}/{epochs} ---")
        train(model, device, train_loader, optimizer, criterion)
        evaluate(model, device, test_loader)

    # 9. 학습된 모델 저장하기
    torch.save(model.state_dict(), "mnist_cnn_model.pth")
    print("Model saved to mnist_cnn_model.pth")